## Transformation & loading section

In [7]:
#import the required libraries
import pandas as pd
from sqlalchemy import create_engine

In [8]:
#reading the required csv files for reading in pandas
csv_file1 = "Resources/phe_deaths_london_boroughs.csv"
csv_file2="Resources/phe_vaccines_age_london_boroughs.csv"

#define the required data frames 
phe_deaths_data_df = pd.read_csv(csv_file1)
phe_vaccines_data_df = pd.read_csv(csv_file2)

In [10]:
#reading the first dataframe
phe_deaths_data_df.head()

,area_name,area_code,date,new_deaths,total_deaths
0,Barking and Dagenham,E09000002,2020-03-09,0,0
1,Barnet,E09000003,2020-03-09,0,0
2,Bexley,E09000004,2020-03-09,0,0
3,Brent,E09000005,2020-03-09,0,0
4,Bromley,E09000006,2020-03-09,0,0


In [11]:
#reading the second dataframe
phe_vaccines_data_df.head()

,area_name,area_code,date,dose,age_band,age_higher,age_lower,cum_doses,new_doses,population,new_prop,cum_prop
0,Barking and Dagenham,E09000002,2020-12-08,1st dose,12 - 15 years,15,12,0,0,13415,0.000000,0.000000
1,Barking and Dagenham,E09000002,2020-12-08,1st dose,16 - 17 years,17,16,0,0,5541,0.000000,0.000000
2,Barking and Dagenham,E09000002,2020-12-08,1st dose,18 - 24 years,24,18,0,0,17719,0.000000,0.000000
3,Barking and Dagenham,E09000002,2020-12-08,1st dose,25 - 29 years,29,25,0,0,15880,0.000000,0.000000
4,Barking and Dagenham,E09000002,2020-12-08,1st dose,30 - 34 years,34,30,1,1,17683,0.000057,0.000057


In [12]:
##doing a bit of cleaning by removing some columns not needed
phe_vaccines_data_df_updated=phe_vaccines_data_df[['area_name','area_code','date','dose','age_band','cum_doses','new_doses','population']].copy()

In [13]:
##listing the columns in the updated dataframe
phe_vaccines_data_df_updated.columns

Index(['area_name', 'area_code', 'date', 'dose', 'age_band', 'cum_doses',
       'new_doses', 'population'],
      dtype='object')

In [14]:
##reading updated the updated dataframe
phe_vaccines_data_df_updated.head()

,area_name,area_code,date,dose,age_band,cum_doses,new_doses,population
0,Barking and Dagenham,E09000002,2020-12-08,1st dose,12 - 15 years,0,0,13415
1,Barking and Dagenham,E09000002,2020-12-08,1st dose,16 - 17 years,0,0,5541
2,Barking and Dagenham,E09000002,2020-12-08,1st dose,18 - 24 years,0,0,17719
3,Barking and Dagenham,E09000002,2020-12-08,1st dose,25 - 29 years,0,0,15880
4,Barking and Dagenham,E09000002,2020-12-08,1st dose,30 - 34 years,1,1,17683


In [19]:
#resetting the index to preparing to load into the SQL
phe_deaths_data_df.reset_index(inplace=True)

In [20]:
#renaming the index to id 
phe_deaths_data_df.rename(columns={'index':'id'}, inplace=True)


In [21]:
#reading dataframe to verify the column name change
phe_deaths_data_df.head()

,id,area_name,area_code,date,new_deaths,total_deaths
0,0,Barking and Dagenham,E09000002,2020-03-09,0,0
1,1,Barnet,E09000003,2020-03-09,0,0
2,2,Bexley,E09000004,2020-03-09,0,0
3,3,Brent,E09000005,2020-03-09,0,0
4,4,Bromley,E09000006,2020-03-09,0,0


In [ ]:
#resetting the index of second dataframe to preparing to load into the SQL
phe_vaccines_data_df_updated.reset_index(inplace=True)

In [ ]:
#renaming the index to id
phe_vaccines_data_df_updated.rename(columns={'index':'id'}, inplace=True)

In [31]:
#readng the dataframe
phe_vaccines_data_df_updated.head()

,id,area_name,area_code,date,dose,age_band,cum_doses,new_doses,population
0,0,Barking and Dagenham,E09000002,2020-12-08,1st dose,12 - 15 years,0,0,13415
1,1,Barking and Dagenham,E09000002,2020-12-08,1st dose,16 - 17 years,0,0,5541
2,2,Barking and Dagenham,E09000002,2020-12-08,1st dose,18 - 24 years,0,0,17719
3,3,Barking and Dagenham,E09000002,2020-12-08,1st dose,25 - 29 years,0,0,15880
4,4,Barking and Dagenham,E09000002,2020-12-08,1st dose,30 - 34 years,1,1,17683


In [32]:
#making a connection to the postgres sql
rds_connection_string = "postgres:postgres@localhost:5432/project2_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [34]:
#displaying the names of the tables created in postgresql
engine.table_names()

['borough_deaths', 'borough_vaccines']

In [38]:
##load data into postgresql
phe_deaths_data_df.to_sql(name='borough_deaths', con=engine, if_exists='append', index=False)

In [39]:
#running a query to see the records loaded from first data frame to correspoding table
pd.read_sql_query('select * from borough_deaths', con=engine).head()

,id,area_name,area_code,date,new_deaths,total_deaths
0,22238,Wandsworth,E09000032,2022-02-01,0,545
1,22239,Westminster,E09000033,2022-02-01,0,371
2,0,Barking and Dagenham,E09000002,2020-03-09,0,0
3,1,Barnet,E09000003,2020-03-09,0,0
4,2,Bexley,E09000004,2020-03-09,0,0


In [ ]:
##load data into postgresql
phe_vaccines_data_df_updated.to_sql(name='borough_vaccines', con=engine, if_exists='append', index=False)

In [ ]:
##running a query to see the records loaded from second data frame to correspoding table
pd.read_sql_query('select * from borough_vaccines', con=engine).head()

In [42]:
#Query to find the area with highest number of doses
pd.read_sql_query('select area_name, age_band, sum(new_doses) as cum_doses from borough_vaccines\
 group by area_name, age_band order by cum_doses desc;', con=engine).head()

,area_name,age_band,cum_doses
0,Wandsworth,25 - 29 years,109349
1,Lambeth,25 - 29 years,100207
2,Tower Hamlets,25 - 29 years,94317
3,Wandsworth,30 - 34 years,92821
4,Tower Hamlets,30 - 34 years,90351


In [44]:
#Query to find the area with highest number of deaths
pd.read_sql_query('select area_name, sum(new_deaths) as num_of_deaths from borough_deaths\
 group by 1 order by 2 desc;', con=engine).head()

,area_name,num_of_deaths
0,Croydon,955
1,Havering,953
2,Barnet,883
3,Redbridge,861
4,Brent,834
